# Final Project
Clay Kindiger and Haoyang Pang

## Load Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math

## Load Data

In [23]:
players_team = pd.read_csv('full_player_stats.csv')
advanced_stats = pd.read_csv('data/advanced_players.csv')
mvp_voting = pd.read_csv('data/mvp_voting_shares.csv')
dpoy_voting = pd.read_csv('data/dpoy_voting.csv')

In [3]:
players_team.head()

,Season,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,L,W/L%,GB,PS/G,PA/G,SRS,Playoffs,Losing_season,mvp,dpoy
0,1998,Mahmoud Abdul-Rauf,PG,28.0,SAC,31.0,0.0,530.0,103.0,273.0,...,55,0.329,34.0,93.1,98.7,-5.83,0,1,0,0
1,1998,Tariq Abdul-Wahad,SG,23.0,SAC,59.0,16.0,959.0,144.0,357.0,...,55,0.329,34.0,93.1,98.7,-5.83,0,1,0,0
2,1998,Terry Dehere,PG,26.0,SAC,77.0,18.0,1410.0,180.0,451.0,...,55,0.329,34.0,93.1,98.7,-5.83,0,1,0,0
3,1998,Lawrence Funderburke,PF,27.0,SAC,52.0,1.0,1094.0,191.0,390.0,...,55,0.329,34.0,93.1,98.7,-5.83,0,1,0,0
4,1998,Derek Grimm,SF,23.0,SAC,9.0,0.0,34.0,4.0,14.0,...,55,0.329,34.0,93.1,98.7,-5.83,0,1,0,0


In [4]:
advanced_stats.head()

,Season,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,...,Unnamed: 19,OWS,DWS,WS,WS/48,Unnamed: 24,OBPM,DBPM,BPM,VORP
0,1998,Mahmoud Abdul-Rauf,PG,28.0,SAC,31.0,530.0,10.5,0.405,0.114,...,NaN,-0.5,0.2,-0.3,-0.025,NaN,-3.2,-1.5,-4.7,-0.4
1,1998,Tariq Abdul-Wahad,SG,23.0,SAC,59.0,959.0,10.1,0.456,0.053,...,NaN,-0.8,0.6,-0.2,-0.008,NaN,-3.6,-1.2,-4.8,-0.7
2,1998,Shareef Abdur-Rahim,SF,21.0,VAN,82.0,2950.0,21.1,0.562,0.038,...,NaN,6.3,0.6,6.9,0.113,NaN,3.4,-1.8,1.6,2.6
3,1998,Cory Alexander,PG,24.0,TOT,60.0,1298.0,15.2,0.548,0.44,...,NaN,1.0,1.2,2.2,0.081,NaN,0.4,0.1,0.5,0.8
4,1998,Cory Alexander,PG,24.0,SAS,37.0,501.0,11.1,0.512,0.441,...,NaN,-0.2,0.8,0.7,0.065,NaN,-2.3,1.1,-1.2,0.1


In [5]:
mvp_voting.head()

,Season,Player,Pts Won,Pts Max,Share
0,1998,Michael Jordan,1084.0,1160,0.934
1,1998,Karl Malone,842.0,1160,0.726
2,1998,Gary Payton,431.0,1160,0.372
3,1998,Shaquille O'Neal,311.0,1160,0.268
4,1998,Tim Duncan,148.0,1160,0.128


In [6]:
dpoy_voting.head()

,Rank,Player,Age,Tm,First,Pts Won,Pts Max,Share,G,MP,...,BLK,FG%,3P%,FT%,WS,WS/48,DWS,DBPM,DRtg,-9999
0,1,Marcus Smart,27,BOS,37.0,257.0,500,0.514,71,32.3,...,0.3,0.418,0.331,0.793,5.6,0.116,3.7,1.7,107,smartma01
1,2,Mikal Bridges,25,PHO,22.0,202.0,500,0.404,82,34.8,...,0.4,0.534,0.369,0.834,8.9,0.150,3.7,1.1,110,bridgmi01
2,3,Rudy Gobert,29,UTA,12.0,136.0,500,0.272,66,32.1,...,2.1,0.713,0.000,0.690,11.7,0.264,4.3,1.7,103,goberru01
3,4,Bam Adebayo,24,MIA,13.0,128.0,500,0.256,56,32.6,...,0.8,0.557,0.000,0.753,7.2,0.188,3.5,2.1,104,adebaba01
4,5,Jaren Jackson Jr.,22,MEM,10.0,99.0,500,0.198,78,27.3,...,2.3,0.415,0.319,0.823,5.4,0.121,3.7,0.8,106,jacksja02


## Clean Data

In [26]:
players_team.drop(columns = ['mvp','dpoy'],inplace = True)

In [57]:
season = []
start = 2023
for idx,row in dpoy_voting.iterrows():
    if row['Rank'] == '1':
        start -= 1
    
    season.append(start)
dpoy_voting['Season'] = season

In [58]:
dpoy_voting = dpoy_voting[['Season', 'Player',  'Pts Won', 'Pts Max', 'Share']]
dpoy_voting.head()

,Season,Player,Pts Won,Pts Max,Share
0,2022,Marcus Smart,257.0,500,0.514
1,2022,Mikal Bridges,202.0,500,0.404
2,2022,Rudy Gobert,136.0,500,0.272
3,2022,Bam Adebayo,128.0,500,0.256
4,2022,Jaren Jackson Jr.,99.0,500,0.198


In [16]:
advanced_stats = advanced_stats[advanced_stats['Player'] != 'Player']

In [24]:
players_teams = {}
for ply,group in advanced_stats.groupby(['Player','Season']):
    if group.shape[0] > 1:
        tm = list(group.Tm)[-1]
        if 28 in group.index:
            print(group.Tm)
        players_teams[list(group.index)[0]] = tm
        if tm == 'TOT':
            print(ply)

28     Tm
57     Tm
82     Tm
105    Tm
130    Tm
155    Tm
178    Tm
201    Tm
222    Tm
247    Tm
270    Tm
291    Tm
316    Tm
339    Tm
364    Tm
387    Tm
416    Tm
453    Tm
480    Tm
517    Tm
544    Tm
Name: Tm, dtype: object


In [25]:
teams_list = []
for idx,row in advanced_stats.iterrows():
    if idx in players_teams.keys():
        tm = players_teams[idx]
        teams_list.append(tm)
    else:
        tm = row['Tm']
        if idx == 3613:
            tm = 'UTA'
        elif idx == 21:
            tm = 'MIA'
        elif idx == 4189:
            tm = 'WAS'
        elif idx == 3037:
            tm = 'BOS'
        elif idx == 11099:
            tm = 'CHI'
        elif idx == 6128:
            tm = 'NJN'
        teams_list.append(tm)
advanced_stats['Tm'] = teams_list

In [46]:
advanced_stats.drop_duplicates(['Player','Season'],keep = 'first',inplace = True)

In [28]:
players_team.shape

(11862, 41)

In [31]:
advanced_stats.shape

(15377, 29)

In [33]:
advanced_stats.columns

Index(['Season', 'Player', 'Pos', 'Age', 'Tm', 'G', 'MP', 'PER', 'TS%', '3PAr',
       'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%',
       'Unnamed: 19', 'OWS', 'DWS', 'WS', 'WS/48', 'Unnamed: 24', 'OBPM',
       'DBPM', 'BPM', 'VORP'],
      dtype='object')

In [34]:
players_team.columns

Index(['Season', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA',
       'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA',
       'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
       'Year', 'Team', 'W', 'L', 'W/L%', 'GB', 'PS/G', 'PA/G', 'SRS',
       'Playoffs', 'Losing_season'],
      dtype='object')

In [38]:
advanced_stats = advanced_stats[['Season', 'Player', 'Tm', 'PER', 'TS%', '3PAr',
       'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%',
       'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP']]

In [47]:
full_players = players_team.merge(advanced_stats, on = ['Player','Tm','Season'],how = 'inner')
full_players.shape

(11860, 61)

In [48]:
full_players.head()

,Season,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP
0,1998,Mahmoud Abdul-Rauf,PG,28.0,SAC,31.0,0.0,530.0,103.0,273.0,...,6.4,25.1,-0.5,0.2,-0.3,-0.025,-3.2,-1.5,-4.7,-0.4
1,1998,Tariq Abdul-Wahad,SG,23.0,SAC,59.0,16.0,959.0,144.0,357.0,...,13.6,22.1,-0.8,0.6,-0.2,-0.008,-3.6,-1.2,-4.8,-0.7
2,1998,Terry Dehere,PG,26.0,SAC,77.0,18.0,1410.0,180.0,451.0,...,16.3,18.6,0.3,0.7,1.0,0.035,-2.5,-0.9,-3.4,-0.5
3,1998,Lawrence Funderburke,PF,27.0,SAC,52.0,1.0,1094.0,191.0,390.0,...,11.8,21.3,1.4,0.7,2.1,0.092,0.9,-1.4,-0.5,0.4
4,1998,Derek Grimm,SF,23.0,SAC,9.0,0.0,34.0,4.0,14.0,...,16.8,23.4,-0.1,0.1,0.0,-0.012,-4.1,1.6,-2.5,0.0


In [55]:
full_players_mvp = full_players.merge(mvp_voting,on = ['Season','Player'],how = 'left')
full_players_mvp.shape

(11860, 64)

In [61]:
full_players_mvp

,Season,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,Pts Won,Pts Max,Share
0,1998,Mahmoud Abdul-Rauf,PG,28.0,SAC,31.0,0.0,530.0,103.0,273.0,...,0.2,-0.3,-0.025,-3.2,-1.5,-4.7,-0.4,NaN,NaN,NaN
1,1998,Tariq Abdul-Wahad,SG,23.0,SAC,59.0,16.0,959.0,144.0,357.0,...,0.6,-0.2,-0.008,-3.6,-1.2,-4.8,-0.7,NaN,NaN,NaN
2,1998,Terry Dehere,PG,26.0,SAC,77.0,18.0,1410.0,180.0,451.0,...,0.7,1.0,0.035,-2.5,-0.9,-3.4,-0.5,NaN,NaN,NaN
3,1998,Lawrence Funderburke,PF,27.0,SAC,52.0,1.0,1094.0,191.0,390.0,...,0.7,2.1,0.092,0.9,-1.4,-0.5,0.4,NaN,NaN,NaN
4,1998,Derek Grimm,SF,23.0,SAC,9.0,0.0,34.0,4.0,14.0,...,0.1,0.0,-0.012,-4.1,1.6,-2.5,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11855,2022,Trevelin Queen,SG,24.0,HOU,10.0,0.0,74.0,15.0,33.0,...,0.1,0.1,0.08,2.7,-0.8,1.9,0.1,NaN,NaN,NaN
11856,2022,Dennis Schröder,PG,28.0,HOU,64.0,29.0,1837.0,312.0,724.0,...,1.9,2.9,0.076,-1.5,-0.4,-1.9,0.0,NaN,NaN,NaN
11857,2022,Alperen Şengün,C,19.0,HOU,72.0,13.0,1489.0,249.0,525.0,...,1.3,2.1,0.068,-1.6,0.6,-1.0,0.4,NaN,NaN,NaN
11858,2022,Jae'Sean Tate,PF,26.0,HOU,78.0,77.0,2056.0,363.0,729.0,...,1.1,3.0,0.071,-1.3,-0.5,-1.8,0.1,NaN,NaN,NaN


In [62]:
full_players_mvp[full_players]

Season         0
Player         0
Pos            0
Age            0
Tm             0
           ...  
BPM            0
VORP           0
Pts Won    11494
Pts Max    11494
Share      11494
Length: 64, dtype: int64

In [59]:
full_players_awards = full_players.merge(dpoy_voting,on = ['Season','Player'],how = 'left',suffixes = ('_mvp','_dpoy'))
full_players_awards.shape

(11860, 64)

In [60]:
full_players_awards

,Season,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,Pts Won,Pts Max,Share
0,1998,Mahmoud Abdul-Rauf,PG,28.0,SAC,31.0,0.0,530.0,103.0,273.0,...,0.2,-0.3,-0.025,-3.2,-1.5,-4.7,-0.4,NaN,NaN,NaN
1,1998,Tariq Abdul-Wahad,SG,23.0,SAC,59.0,16.0,959.0,144.0,357.0,...,0.6,-0.2,-0.008,-3.6,-1.2,-4.8,-0.7,NaN,NaN,NaN
2,1998,Terry Dehere,PG,26.0,SAC,77.0,18.0,1410.0,180.0,451.0,...,0.7,1.0,0.035,-2.5,-0.9,-3.4,-0.5,NaN,NaN,NaN
3,1998,Lawrence Funderburke,PF,27.0,SAC,52.0,1.0,1094.0,191.0,390.0,...,0.7,2.1,0.092,0.9,-1.4,-0.5,0.4,NaN,NaN,NaN
4,1998,Derek Grimm,SF,23.0,SAC,9.0,0.0,34.0,4.0,14.0,...,0.1,0.0,-0.012,-4.1,1.6,-2.5,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11855,2022,Trevelin Queen,SG,24.0,HOU,10.0,0.0,74.0,15.0,33.0,...,0.1,0.1,0.08,2.7,-0.8,1.9,0.1,NaN,NaN,NaN
11856,2022,Dennis Schröder,PG,28.0,HOU,64.0,29.0,1837.0,312.0,724.0,...,1.9,2.9,0.076,-1.5,-0.4,-1.9,0.0,NaN,NaN,NaN
11857,2022,Alperen Şengün,C,19.0,HOU,72.0,13.0,1489.0,249.0,525.0,...,1.3,2.1,0.068,-1.6,0.6,-1.0,0.4,NaN,NaN,NaN
11858,2022,Jae'Sean Tate,PF,26.0,HOU,78.0,77.0,2056.0,363.0,729.0,...,1.1,3.0,0.071,-1.3,-0.5,-1.8,0.1,NaN,NaN,NaN


## Create Models

## Evaluate Models